```
; parinfer-fixer.lisp - Fix unbalanced parentheses in LLM-generated Lisp code
;
; Copyright (C) 2025
;
; License: See LICENSE file
;
; This module uses parinfer-rust to fix common parenthesis errors in
; LLM-generated ACL2/Common Lisp code. LLMs often produce syntactically
; valid-looking code with incorrect indentation-to-paren correspondence.
;
; Parinfer's "indent mode" infers parentheses from indentation, which is
; exactly what we need since LLMs usually get indentation right but parens wrong.
```

In [1]:
(in-package "ACL2")

 "ACL2"


In [2]:
(include-book "std/util/bstar" :dir :system)


Summary
Form:  ( INCLUDE-BOOK "std/util/bstar" ...)
Rules: NIL
Time:  0.04 seconds (prove: 0.00, print: 0.00, other: 0.04)
 "/home/acl2/books/std/util/bstar.lisp"


In [3]:
(defttag :parinfer-fixer)


TTAG NOTE: Adding ttag :PARINFER-FIXER from the top level loop.
 :PARINFER-FIXER


In [4]:
;; Path to parinfer-rust executable (must be in PATH after sourcing cargo env)
;; We'll call it via shell since there's no direct FFI
(defconst *parinfer-rust-cmd* "parinfer-rust")


Summary
Form:  ( DEFCONST *PARINFER-RUST-CMD* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *PARINFER-RUST-CMD*


In [5]:
;; Mode for parinfer:
;;   "indent" - Infer parens from indentation (best for LLM output)
;;   "paren"  - Infer indentation from parens
;;   "smart"  - Try to be smart about what to fix
(defconst *parinfer-default-mode* "indent")


Summary
Form:  ( DEFCONST *PARINFER-DEFAULT-MODE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *PARINFER-DEFAULT-MODE*


In [6]:
;; Language option for Common Lisp / ACL2 (enables #| |# block comments)
(defconst *parinfer-lisp-options* "--lisp-block-comments")


Summary
Form:  ( DEFCONST *PARINFER-LISP-OPTIONS* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *PARINFER-LISP-OPTIONS*


In [7]:
(defun parens-balanced-simple-aux (chars parens brackets braces)
  (declare (xargs :mode :program))
  (if (endp chars)
      (and (= parens 0) (= brackets 0) (= braces 0))
    (let ((ch (car chars)))
      (cond
        ((eql ch #\() (parens-balanced-simple-aux (cdr chars) (1+ parens) brackets braces))
        ((eql ch #\)) (if (> parens 0)
                          (parens-balanced-simple-aux (cdr chars) (1- parens) brackets braces)
                        nil)) ; Unmatched close paren
        ((eql ch #\[) (parens-balanced-simple-aux (cdr chars) parens (1+ brackets) braces))
        ((eql ch #\]) (if (> brackets 0)
                          (parens-balanced-simple-aux (cdr chars) parens (1- brackets) braces)
                        nil))
        ((eql ch #\{) (parens-balanced-simple-aux (cdr chars) parens brackets (1+ braces)))
        ((eql ch #\}) (if (> braces 0)
                          (parens-balanced-simple-aux (cdr chars) parens brackets (1- braces))
                        nil))
        (t (parens-balanced-simple-aux (cdr chars) parens brackets braces))))))


Summary
Form:  ( DEFUN PARENS-BALANCED-SIMPLE-AUX ...)
Rules: NIL
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
 PARENS-BALANCED-SIMPLE-AUX


In [8]:
;; Check if parentheses are balanced in a string
;; Returns t if balanced, nil otherwise
;; Note: This is a simple check that doesn't handle strings/comments
(defun parens-balanced-simple (str)
  (declare (xargs :mode :program))
  (let ((chars (coerce str 'list)))
    (parens-balanced-simple-aux chars 0 0 0)))


Summary
Form:  ( DEFUN PARENS-BALANCED-SIMPLE ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 PARENS-BALANCED-SIMPLE


In [9]:
;; Execute a shell command with input string and return output
;; Returns (mv error-string output-string state)
(defun run-shell-command-with-input (cmd input-str state)
  (declare (xargs :mode :program :stobjs state))
  (b* (;; Write input to a temp file
       (temp-in "/tmp/parinfer-input.lisp")
       (temp-out "/tmp/parinfer-output.lisp")
       ;; Write the input
       ((mv channel state)
        (open-output-channel temp-in :character state))
       ((when (not channel))
        (mv "Failed to open temp input file" "" state))
       (state (princ$ input-str channel state))
       (state (close-output-channel channel state))
       ;; Run the command: cmd < temp-in > temp-out
       (full-cmd (concatenate 'string cmd " < " temp-in " > " temp-out " 2>&1"))
       ;; sys-call+ returns (mv erp val state) - 3 values
       ((mv exit-code ?cmd-output state) (sys-call+ "sh" (list "-c" full-cmd) state))
       ;; read-file-into-string2 returns just a string (not mv)
       (output (read-file-into-string2 temp-out 0 nil :default state)))
    (if (and (not exit-code) output)
        (mv nil output state)
      (mv (concatenate 'string "Command failed with exit code: " 
                       (coerce (explode-atom (or exit-code -1) 10) 'string))
          (or output "") state))))


Summary
Form:  ( DEFUN RUN-SHELL-COMMAND-WITH-INPUT ...)
Rules: NIL
Time:  0.03 seconds (prove: 0.00, print: 0.00, other: 0.03)
 RUN-SHELL-COMMAND-WITH-INPUT


In [10]:
;; Fix Lisp code using parinfer-rust indent mode
;; This infers correct parentheses from the code's indentation
;; Returns (mv error-string fixed-code state)
(defun parinfer-fix-code (code state)
  (declare (xargs :mode :program :stobjs state))
  ;; Source cargo env to ensure parinfer-rust is in PATH
  (let* ((cmd (concatenate 'string 
                           ". \"$HOME/.cargo/env\" && "
                           *parinfer-rust-cmd* 
                           " -m " *parinfer-default-mode*
                           " " *parinfer-lisp-options*)))
    (run-shell-command-with-input cmd code state)))


Summary
Form:  ( DEFUN PARINFER-FIX-CODE ...)
Rules: NIL
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
 PARINFER-FIX-CODE


In [11]:
;; Fix code with explicit mode selection
;; mode should be "indent", "paren", or "smart"
(defun parinfer-fix-code-with-mode (code mode state)
  (declare (xargs :mode :program :stobjs state))
  ;; Source cargo env to ensure parinfer-rust is in PATH
  (let* ((cmd (concatenate 'string 
                           ". \"$HOME/.cargo/env\" && "
                           *parinfer-rust-cmd* 
                           " -m " mode
                           " " *parinfer-lisp-options*)))
    (run-shell-command-with-input cmd code state)))


Summary
Form:  ( DEFUN PARINFER-FIX-CODE-WITH-MODE ...)
Rules: NIL
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
 PARINFER-FIX-CODE-WITH-MODE


In [12]:
;; Check and optionally fix LLM-generated code
;; Returns (mv was-fixed error fixed-code state)
;; was-fixed is t if code was modified, nil if already correct
(defun check-and-fix-lisp-code (code state)
  (declare (xargs :mode :program :stobjs state))
  (if (parens-balanced-simple code)
      ;; Code looks balanced, but let parinfer verify structure
      (b* (((mv err fixed state) (parinfer-fix-code code state)))
        (if err
            (mv nil err code state)
          (mv (not (equal code fixed)) nil fixed state)))
    ;; Code is definitely unbalanced, fix it
    (b* (((mv err fixed state) (parinfer-fix-code code state)))
      (if err
          (mv nil err code state)
        (mv t nil fixed state)))))


Summary
Form:  ( DEFUN CHECK-AND-FIX-LISP-CODE ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 CHECK-AND-FIX-LISP-CODE


In [13]:
;; Fix code and report what changed
;; Returns (mv report fixed-code state)
(defun fix-lisp-code-with-report (code state)
  (declare (xargs :mode :program :stobjs state))
  (b* (((mv was-fixed err fixed state) (check-and-fix-lisp-code code state)))
    (cond
      (err (mv (concatenate 'string "[Parinfer Error] " err) code state))
      (was-fixed (mv "[Parinfer] Fixed unbalanced parentheses" fixed state))
      (t (mv nil code state)))))


Summary
Form:  ( DEFUN FIX-LISP-CODE-WITH-REPORT ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 FIX-LISP-CODE-WITH-REPORT


In [14]:
;; Test parinfer on a code string (for interactive testing)
(defmacro test-parinfer (code)
  `(make-event
    (mv-let (err result state)
      (parinfer-fix-code ,code state)
      (if err
          (prog2$ (cw "Error: ~s0~%" err)
                  (mv nil '(value-triple :error) state))
        (prog2$ (cw "Result:~%~s0~%" result)
                (mv nil '(value-triple :ok) state))))))


Summary
Form:  ( DEFMACRO TEST-PARINFER ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TEST-PARINFER


Another cell that acl2-kernel hangs on.

In [ ]:
;; Example test cases showing common LLM errors
(defconst *test-cases*
  '("(defun foo (x)\n  (+ x 1"
    "(defun bar (x)\n  (+ x 1)))"
    "(defun baz (x)\n  (+ x 1))"
    "(let ((a 1)\n      (b 2))\n  (+ a b"
    "(defun f1 (x) x)\n(defun f2 (y) y"
    ))

In [18]:
*test-cases*

("(defun foo (x)n  (+ x 1" "(defun bar (x)n  (+ x 1)))"
                           "(defun baz (x)n  (+ x 1))"
                           "(let ((a 1)n      (b 2))n  (+ a b"
                           "(defun f1 (x) x)n(defun f2 (y) y")
